<a href="https://colab.research.google.com/github/Marlrero/AI_PrimaryEducation/blob/main/housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>